# Readability

In [1]:
import re
import jieba
import pandas as pd
import numpy as np
import json
from nltk.probability import *
import math

In [2]:
# Prepare Data
all_data = pd.DataFrame()
for i in range(17):
    i += 1
    path = "data/record" + str(i) + ".xls"
    all_data = all_data.append(pd.read_excel(path), sort=False)
data = all_data[['文档号码','投资者关系活动主要内容介绍']]
print('Before cleaning:')
print(data.shape)
data = data.dropna()
print('After cleaning:')
print(data.shape)
print(data.head(10))

Before cleaning:
(58611, 2)
After cleaning:
(41986, 2)
          文档号码                                      投资者关系活动主要内容介绍
0   1200573239  一、问题回答。\n1、非公开发行的进展情况如何？\n答：目前中国证监会正在审核公司非公开发行...
1   1200573241  会议主要关注BE部分。\n\n问题一：BE最近有何变化？\n回答：公司目前业务进展顺利，业务...
2   1200573242  会议主要关注BYD部分。\n\n问题一：公司新能源汽车的目前的进展情况？\n回答：公司将新能...
3   1200573245  问：请您谈谈公司2015年的收入季节性是否还会存在？\n答：公司2013、2014年的第三季...
5   1200573285  1、公司目前的出口情况？\n答：近几年来，公司加强了海外出口的投入力度，组建了专门的国际贸易...
8   1200573318  国联证券研究员于2015年01月22日到公司化工新材料产业园区进行现场调研座谈，主要内容如下...
9   1200573672  1.公司机床的优势？\n公司具有规模优势，公司是国内机床产品种类最为齐全的机床类企业。具备提...
10  1200573678  一、签署《调研来访承诺书》；\n二、回答调研提问，主要问答简述如下：\n1、问：公司规划建新...
11  1200573726  一、交流内容\n董事会秘书：欢迎英大基金来访本公司。投资者和研究人员来访本公司须遵循深圳交易...
12  1200573767  1、本次收购方式、交易价格及收购资金来源？\n答：公司全资子公司翼鹏投资、公司及公司实际控制...


## Part 1. Preprocessing
### 1.1 Cleaning

In [3]:
# process the error docs after second-level cleaning
def clean(doc):
    chi = r'([\u4E00-\u9FA5]|[0-9]|[“”、。《》！，：；？\.%])'
    pa = re.compile(chi)
    return "".join(re.findall(pa, doc))

In [4]:
print(clean(data.iloc[10][1]))
print("--------------")
print(data.iloc[10][1])

1.公司基本经营情况公司专业从事集成电路封装测试业务，目前公司拥有500多种产品，业务范围包括微处理器、数字电路、模拟电路、数模混合电路、射频电路等产品的封测服务。公司在中高端产品的封装技术上具有优势，目前、、和等产品是公司的主打产品，从2014年4季度开始，、和等先进封装产品也即将进入大规模生产阶段。半导体行业是一个波动性较大、周期性较明显的行业，目前行业比较景气，同时国家出台产业基金扶持政策，行业景气周期与国家扶持政策周期相叠加，未来市场前景比较光明。2.目前公司的产品结构公司高引脚系列产品占比约40%，低引脚系列产品占比约40%，系列产品占比约20%。高引脚系列产品主要包括、、等。3.公司的核心竞争优势1丰富的国际市场开发经验和较高的客户服务质量；2领先的封装技术水平和科技研发实力；3同行业领先的管理能力4.公司定增项目进展情况公司已向证监会提交申请材料，证监会认为公司申请材料齐全、符合法定形式，予以受理，并出具《中国证监会行政许可申请受理通知书》，目前正在进一步审核当中。2015年1月22日，公司披露了《关于非公开发行股票相关事项的公告》，针对本次募集资金投资项目的可预见风险、公司补充流动资金的必要性及合理性、政府补助等优惠政策发生变化的风险等六项内容，进行了说明。
--------------
1.公司基本经营情况
公司专业从事集成电路封装测试业务，目前公司拥有500多种产品，业务范围包括微处理器、数字电路、模拟电路、数模混合电路、射频电路等产品的封测服务。
公司在中高端产品的封装技术上具有优势，目前 BGA、QFP/LQFP、QFN和POWER等产品是公司的主打产品，从2014年4季度开始，WLP、BUMP和FC等先进封装产品也即将进入大规模生产阶段。半导体行业是一个波动性较大、周期性较明显的行业，目前行业比较景气，同时国家出台产业基金扶持政策，行业景气周期与国家扶持政策周期相叠加，未来市场前景比较光明。

2.目前公司的产品结构
公司高引脚系列产品占比约40%，低引脚系列产品占比约40%，POWER系列产品占比约20%。高引脚系列产品主要包括BGA、QFP/LQFP、QFN等。

3.公司的核心竞争优势
 （1）丰富的国际市场开发经验和较高的客户服务质量；
 （2）领先的封装技术水平和科技研发实力；
 （3）同行业领先的管理能力

4.公司定增项目进展

In [5]:
seg_list = jieba.cut(clean(data.iloc[10][1][0:500]), cut_all=False, HMM=True)
print("Default Mode: " + "/ ".join(seg_list))  # 默认模式

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MSI\AppData\Local\Temp\jieba.cache
Loading model cost 0.705 seconds.
Prefix dict has been built succesfully.


Default Mode: 1/ ./ 公司/ 基本/ 经营/ 情况/ 公司/ 专业/ 从事/ 集成电路/ 封装/ 测试/ 业务/ ，/ 目前/ 公司/ 拥有/ 500/ 多种产品/ ，/ 业务范围/ 包括/ 微处理器/ 、/ 数字电路/ 、/ 模拟/ 电路/ 、/ 数模/ 混合/ 电路/ 、/ 射频/ 电路/ 等/ 产品/ 的/ 封测/ 服务/ 。/ 公司/ 在/ 中/ 高端/ 产品/ 的/ 封装/ 技术/ 上/ 具有/ 优势/ ，/ 目前/ 、/ 、/ 和/ 等/ 产品/ 是/ 公司/ 的/ 主打产品/ ，/ 从/ 2014/ 年/ 4/ 季度/ 开始/ ，/ 、/ 和/ 等/ 先进/ 封装/ 产品/ 也/ 即将/ 进入/ 大规模/ 生产/ 阶段/ 。/ 半导体/ 行业/ 是/ 一个/ 波动性/ 较大/ 、/ 周期性/ 较/ 明显/ 的/ 行业/ ，/ 目前/ 行业/ 比较/ 景气/ ，/ 同时/ 国家/ 出台/ 产业/ 基金/ 扶持/ 政策/ ，/ 行业/ 景气/ 周期/ 与/ 国家/ 扶持/ 政策/ 周期/ 相/ 叠加/ ，/ 未来/ 市场前景/ 比较/ 光明/ 。/ 2/ ./ 目前/ 公司/ 的/ 产品/ 结构/ 公司/ 高/ 引脚/ 系列产品/ 占/ 比约/ 40%/ ，/ 低/ 引脚/ 系列产品/ 占/ 比约/ 40%/ ，/ 系列产品/ 占/ 比约/ 20%/ 。/ 高/ 引脚/ 系列产品/ 主要/ 包括/ 、/ 、/ 等/ 。/ 3/ ./ 公司/ 的/ 核心/ 竞争/ 优势/ 1/ 丰富/ 的/ 国际/ 市场/ 开发/ 经验/ 和/ 较/ 高/ 的/ 客户服务/ 质量/ ；/ 2/ 领先/ 的/ 封装/ 技术水平/ 和/ 科技/ 研发/ 实力/ ；/ 3/ 同行业/ 领先/ 的/ 管理/ 能力/ 4/ ./ 公司/ 定增/ 项目/ 进展/ 情况/ 公司/ 已/ 向/ 证监会/ 提交/ 申请材料/ ，/ 证监会/ 认为/ 公司/ 申请材料/ 齐全/ 、/ 符合/ 法定/ 形式/ ，/ 予以/ 受理/ ，/ 并/ 出具/ 《/ 中国证监会/ 行政许可/ 申请/ 受理/ 通


### 1.2 Split into sentences

In [47]:
def sent_seg(cleaned_doc):
    sent_pa = re.compile(r'.+?[？。！]')
    return re.findall(sent_pa, cleaned_doc)

def pure_sent(sent):
    cleaned_sent_pa = re.compile(r'([\u4E00-\u9FA5])')
    return ''.join(re.findall(cleaned_sent_pa, sent))
        
# Size of a doc is defined as the total number of valid Chinese characters
def raw_process(doc):
    cleaned_doc = clean(doc)
    sents = sent_seg(cleaned_doc)
    if not cleaned_doc or not len(sents):
        return {
            'sents': [],
            'size': 0,
            'avg_sent_len' : 0
        }
    else:
        total_length = sum([len(pure_sent(sent)) for sent in sents])
        avg_sent_length = total_length / len(sents)
        return {
            'sents': sents,
            'size' : total_length,
            'avg_sent_len' : avg_sent_length 
        }
    
raw_process(data.iloc[0][1])

{'sents': ['一、问题回答。',
  '1、非公开发行的进展情况如何？',
  '答：目前中国证监会正在审核公司非公开发行项目，公司已完成证监会反馈意见书面回复。',
  '2、新星控股..的基本情况答：新星控股主要从事精密金属零件制造、模具制造、机械装配以及模切加工等业务，产品主要用于硬盘、汽车电子、家用电器、消费电子等行业。',
  '新星控股及其控股子公司是具有丰富生产、研发经验的精密金属零件制造商。',
  '新星控股总部位于新加坡，主要工厂设立于中国苏州和泰国，其客户包括世界知名的硬盘生产商希捷、汽车配件制造商博世、家电制造商飞利浦等。',
  '新星控股的主要产品可以分为两大类：1信息存储硬盘相关精密金属零件；2非硬盘精密金属零件。',
  '信息存储硬盘是广泛运用于企业存储服务器、云存储服务器、台式电脑、笔记本电脑、移动存储设备、数字机顶盒的的存储媒介，目前分为、和混合硬盘。',
  '新星控股生产的信息存储硬盘相关精密金属零件主要是应用于和混合硬盘的硬盘顶盖、磁盘夹具、磁盘分离器等。',
  '非硬盘精密金属零件的用途更加广泛，在汽车零部件、家用电器、工业、消费电子等产品中均有尺寸小但是对精密度要求非常高的金属零件。',
  '在汽车零部件领域，新星控股主要客户为博世。',
  '3、单个硬盘上新星控股精密金属零件的用量？',
  '答：硬盘主要包括机械硬盘，也称为传统硬盘、固态硬盘和混合硬盘。',
  '目前，精密金属零件主要应用在产品上，应用方案最为成熟，而在和市场上相对较小，因此，其市场容量主要取决于的市场容量。',
  '上的精密金属零件主要包括硬盘顶盖、磁盘夹具和磁盘分离器。',
  '单块需要1个硬盘顶盖，1个磁盘夹具，磁盘分离器的数量取决于硬盘内磁盘的数量，如果里仅有一张磁盘，则不需要磁盘分离器，若有2张以上的磁盘，则需要同等数量的磁盘分离器。',
  '目前一张磁盘的容量多为500或1，在市场上主流的容量限制下，一块内约有18张磁盘不等，且以14张为主，因此有24块磁盘分离器。',
  '4、安洁与新星控股整合计划？',
  '答：客户资源和技术资源的整合是安洁科技和新星控股整合的重要方面，安洁科技会利用公司与消费电子品牌客户良好合作关系的客户资源，新星控股会利用新星控股在金属精密件上的技术优势，双方一起参入到消费电子品牌客

### 1.3 Stop word list

In [7]:
with open('stop_words/中文停用词表.txt', 'r', encoding='UTF-8-sig') as f:
    stop_words = [ word.strip().replace('\n', '') for word in f.readlines()]
symbols = stop_words[0:26]
stop_words

['$',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '?',
 '_',
 '“',
 '”',
 '、',
 '。',
 '《',
 '》',
 '！',
 '，',
 '：',
 '；',
 '？',
 '.',
 '%',
 '一',
 '一些',
 '一何',
 '一切',
 '一则',
 '一方面',
 '一旦',
 '一来',
 '一样',
 '一般',
 '一转眼',
 '万一',
 '上',
 '上下',
 '下',
 '不',
 '不仅',
 '不但',
 '不光',
 '不单',
 '不只',
 '不外乎',
 '不如',
 '不妨',
 '不尽',
 '不尽然',
 '不得',
 '不怕',
 '不惟',
 '不成',
 '不拘',
 '不料',
 '不是',
 '不比',
 '不然',
 '不特',
 '不独',
 '不管',
 '不至于',
 '不若',
 '不论',
 '不过',
 '不问',
 '与',
 '与其',
 '与其说',
 '与否',
 '与此同时',
 '且',
 '且不说',
 '且说',
 '两者',
 '个',
 '个别',
 '临',
 '为',
 '为了',
 '为什么',
 '为何',
 '为止',
 '为此',
 '为着',
 '乃',
 '乃至',
 '乃至于',
 '么',
 '之',
 '之一',
 '之所以',
 '之类',
 '乌乎',
 '乎',
 '乘',
 '也',
 '也好',
 '也罢',
 '了',
 '二来',
 '于',
 '于是',
 '于是乎',
 '云云',
 '云尔',
 '些',
 '亦',
 '人',
 '人们',
 '人家',
 '什么',
 '什么样',
 '今',
 '介于',
 '仍',
 '仍旧',
 '从',
 '从此',
 '从而',
 '他',
 '他人',
 '他们',
 '以',
 '以上',
 '以为',
 '以便',
 '以免',
 '以及',
 '以故',
 '以期',
 '以来',
 '以至',
 '以至于',
 '以致',
 '们',
 '任',
 '任何',
 '任凭',
 '似的',
 '但',
 '但凡',
 '但是',
 '何',
 '何以',
 '何况

### 1.4 Generating frequency distribution 
(using stop word list)

In [8]:
def gen_freq_dist(doc):
    stat = raw_process(doc)
    sents = stat['sents']
    freq_dist = dict()
    pa = re.compile(r'([$0123456789?_“”、。《》！，：；？\.%])')
    for sent in sents:
        # calculate sent length after
        words = jieba.cut(sent, cut_all=False, HMM=True)
        for word in words:
            # ignore all the stop words
            if (not word in stop_words) and (not re.findall(pa, word)):
                if not word in freq_dist.keys():
                    freq_dist[word] = 1
                else:
                    freq_dist[word] += 1
    return { 'freq_dist' : freq_dist, 
             'size' : stat['size'],
             'avg_sent_len' : stat['avg_sent_len'],
             'n_sents' : len(sents)
           }

gen_freq_dist(data.iloc[0][1])

{'freq_dist': {'问题': 1,
  '回答': 1,
  '非': 4,
  '公开': 2,
  '发行': 2,
  '进展': 1,
  '情况': 4,
  '答': 5,
  '目前': 4,
  '中国证监会': 1,
  '正在': 1,
  '审核': 1,
  '公司': 16,
  '项目': 1,
  '完成': 3,
  '证监会': 1,
  '反馈': 1,
  '意见': 1,
  '书面': 1,
  '回复': 1,
  '新星': 21,
  '控股': 22,
  '基本': 1,
  '主要': 12,
  '从事': 1,
  '精密': 14,
  '金属': 15,
  '零件': 13,
  '制造': 2,
  '模具': 1,
  '机械': 2,
  '装配': 1,
  '模切': 1,
  '加工': 1,
  '业务': 2,
  '产品': 9,
  '用于': 1,
  '硬盘': 19,
  '汽车': 5,
  '电子': 8,
  '家用电器': 2,
  '消费': 6,
  '行业': 2,
  '子公司': 1,
  '具有': 1,
  '丰富': 2,
  '生产': 3,
  '研发': 2,
  '经验': 1,
  '制造商': 3,
  '总部': 1,
  '位于': 1,
  '新加坡': 1,
  '工厂': 1,
  '设立': 1,
  '中国': 1,
  '苏州': 1,
  '泰国': 1,
  '客户': 9,
  '包括': 3,
  '世界': 1,
  '知名': 1,
  '生产商': 2,
  '希捷': 1,
  '汽车配件': 1,
  '博世': 3,
  '家电': 1,
  '飞利浦': 1,
  '分为': 2,
  '两大类': 1,
  '信息': 6,
  '存储': 6,
  '相关': 3,
  '广泛': 2,
  '运用': 1,
  '企业': 1,
  '服务器': 2,
  '云': 1,
  '台式': 1,
  '电脑': 1,
  '笔记本电脑': 1,
  '移动': 1,
  '存储设备': 2,
  '数字': 1,
  '机顶盒': 1,
  '媒介': 1,
  '混合': 3,
  '应

## Part 2 Compute the readability in different ways

### 2.1 Grade and Semester 
(using common word list)

In [9]:
ch_pa = re.compile(r'([\u4E00-\u9FA5])')
with open('common_words/常用词.txt', 'r', encoding='UTF-8-sig') as f:
    common_words = [ ''.join(re.findall(ch_pa, line)) for line in f.readlines()]
print(len(common_words))
print('为了' in common_words)

56064
True


$\begin{aligned}
年级 &= 17.52547988 + 0.00242523 \times 课文长度 \\
      &+ 0.04414527 \times 平均句长 - 18.33435443 * 常用字比率
\end{aligned}$

$\begin{aligned}
学期 &= 34.53858379 + 0.00491625 \times 课文长度 \\
      &+ 0.08996394 \times 平均句长 - 36.73710603 * 常用字比率
\end{aligned}$

In [38]:
def grade_n_semester(doc):
    stat = gen_freq_dist(doc)
    freq_dist = stat['freq_dist']
    n_words = sum(freq_dist.values())
    
    # compute the percentage of common words
    n_common_words = 0
    for word in freq_dist.keys():
        if word in common_words:
            n_common_words += freq_dist[word]
    
    # all the required statistics
    article_length = stat['size']
    avg_sent_len = stat['avg_sent_len']
    
    # for empty doc
    if not article_length or not avg_sent_len:
        return {
            'grade' : None,
            'semester' : None,
            'common_words_percentage' : None
        }
    
    percent_common_words = n_common_words / n_words
    
    # compute grade & semester
    grade = 17.52547988 + 0.00242523 * article_length \
            + 0.04414527 * avg_sent_len - 18.33435443 * percent_common_words
    semester = 34.53858379 + 0.00491625 * article_length \
             + 0.08996394 * avg_sent_len - 36.73710603 * percent_common_words
    
    return {
        'grade' : grade,
        'semester' : semester,
        'common_words_percentage' : percent_common_words
    }

grade_n_semester(data.iloc[30][1])

{'grade': 8.696122698164126,
 'semester': 16.980809349729793,
 'common_words_percentage': 0.7504725897920604}

### 2.2 Fog value and its modified version 
(using the full frequency distribution (all_freq_dist) and the document frequency distribution (df_dist))

In [11]:
# One-time block
def init_all_freq_dist():
    all_freq_dist = dict()
    count = 0
    for index, d in data.iterrows():
        print('[' + str(count) + '] Processing document ' + str(d['文档号码']) + '...')
        fd = gen_freq_dist(d[1])['freq_dist']
        for k in fd.keys():
            if k not in all_freq_dist.keys():
                all_freq_dist[k] = fd[k]
            else:
                all_freq_dist[k] += fd[k]
        count += 1
    return all_freq_dist

In [12]:
with open('all_freq_dist.json', 'r', encoding='UTF-8-sig') as f:
    all_freq_dist = json.load(f)

all_freq_dist_df = pd.DataFrame.from_dict(all_freq_dist, orient='index', columns=['freq'])
print('Most frequent word is: ' + str(np.argmax(all_freq_dist_df['freq'])))
all_freq_dist_df.describe()

Most frequent word is: 公司


C:\Users\MSI\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


,freq
count,128918.000000
mean,137.010472
std,2841.917420
min,1.000000
25%,1.000000
50%,3.000000
75%,13.000000
max,770176.000000


Given the summary here, apparently defining 5% or 10% as rare/complex words is not appropriate, so instead, define complex words to be frequency less than 1 or 3.

In [13]:
def is_complex(word, threshold=1):
    global all_freq_dist
    return all_freq_dist[word] <= threshold 

In [14]:
# One-time block
def init_df_dist():
    df_dist = dict()
    count = 0
    for index, d in data.iterrows():
        print('[' + str(count) + '] Processing document ' + str(d['文档号码']) + '...')
        fd = gen_freq_dist(d[1])['freq_dist']
        for k in fd.keys():
            df_dist.setdefault(k, 0)
            df_dist[k] += 1
        count += 1
    return df_dist

In [16]:
with open('df_dist.json', 'r', encoding='UTF-8-sig') as f:
    df_dist = json.load(f)

df_dist_df = pd.DataFrame.from_dict(df_dist, orient='index', columns=['df'])
print('Most diverse word is: ' + str(np.argmax(df_dist_df['df'])))
df_dist_df.describe()

Most diverse word is: 公司


,df
count,128918.000000
mean,72.656278
std,671.803797
min,1.000000
25%,1.000000
50%,2.000000
75%,10.000000
max,40252.000000


In [17]:
def weight_of_word(word):
    global data, df_dist
    N = data.shape[0]
    df = df_dist[word]
    return np.log(N / df) / np.log(N)

In [18]:
def fog(word_per_sent, percent_cw):
    return 0.4 * (word_per_sent + 100 * percent_cw)

def fog_all(doc):
    global all_freq_dist
    stat = gen_freq_dist(doc)
    freq_dist = stat['freq_dist']
    n_words = sum(freq_dist.values())
    n_sents = stat['n_sents']
    
    # for empty doc
    if not n_sents or not n_words:
        return {
            'original_fog_t1' : None,
            'original_fog_t3' : None,
            'weighted_fog_t1' : None,
            'weighted_fog_t3' : None
        }
    
    # complex words percentage
    percent_cw_dict = {
        'o_t1' : (sum([freq_dist[word] for word in freq_dist.keys() if is_complex(word, threshold=1)]) / n_words),
        'o_t3' : (sum([freq_dist[word] for word in freq_dist.keys() if is_complex(word, threshold=3)]) / n_words),
        'w_t1' : (sum([(weight_of_word(word) * freq_dist[word]) \
                               for word in freq_dist.keys() if is_complex(word, threshold=1)]) / n_words),
        'w_t3' : (sum([(weight_of_word(word) * freq_dist[word]) \
                               for word in freq_dist.keys() if is_complex(word, threshold=3)]) / n_words)
    }
    
    word_per_sent = n_words / n_sents
    
    return {
            'original_fog_t1' : fog(word_per_sent, percent_cw_dict['o_t1']),
            'original_fog_t3' : fog(word_per_sent, percent_cw_dict['o_t3']),
            'weighted_fog_t1' : fog(word_per_sent, percent_cw_dict['w_t1']),
            'weighted_fog_t3' : fog(word_per_sent, percent_cw_dict['w_t3'])
    }

### 2.3 Computation

In [52]:
all_scores = pd.DataFrame(columns=['文档号码', '总长度', '平均句长', '常用词比例', '年级', '学期', 'original_fog_t1', 'original_fog_t3', 'weighted_fog_t1', 'weighted_fog_t3'])
count = 0
error_docs = []
for index, d in data.iterrows():
    print('[' + str(count) + '] Processing document ' + str(d['文档号码']) + '...')
    count += 1
    doc = d['投资者关系活动主要内容介绍']
    doc_num = d['文档号码']
    scores = pd.DataFrame(columns=['文档号码', '总长度', '平均句长', '常用词比例', '年级', '学期', 'original_fog_t1', 'original_fog_t3', 'weighted_fog_t1', 'weighted_fog_t3'])
    scores['文档号码'] = [doc_num]
    try:
        raw_stat = raw_process(doc)
        scores['总长度'] = [raw_stat['size']]
        scores['平均句长'] = [raw_stat['avg_sent_len']]
        # 学期,年级
        result_1 = grade_n_semester(doc)
        scores['常用词比例'] = [result_1['common_words_percentage']]
        scores['年级'] = [result_1['grade']]
        scores['学期'] = [result_1['semester']]
        # fog
        result_2 = fog_all(doc)
        scores['original_fog_t1'] = [result_2['original_fog_t1']]
        scores['original_fog_t3'] = [result_2['original_fog_t3']]
        scores['weighted_fog_t1'] = [result_2['weighted_fog_t1']]
        scores['weighted_fog_t3'] = [result_2['weighted_fog_t3']]
        # appending
        all_scores = all_scores.append(scores)
    except:
        error_docs.append(doc_num)
        print('An error happened when processing document ' + str(doc_num))

[0] Processing document 1200573239...
[1] Processing document 1200573241...
[2] Processing document 1200573242...
[3] Processing document 1200573245...
[4] Processing document 1200573285...
[5] Processing document 1200573318...
[6] Processing document 1200573672...
[7] Processing document 1200573678...
[8] Processing document 1200573726...
[9] Processing document 1200573767...
[10] Processing document 1200573841...
[11] Processing document 1200573903...
[12] Processing document 1200574029...
[13] Processing document 1200574051...
[14] Processing document 1200574620...
[15] Processing document 1200574888...
[16] Processing document 1200576676...
[17] Processing document 1200579053...
[18] Processing document 1200579060...
[19] Processing document 1200579151...
[20] Processing document 1200579217...
[21] Processing document 1200579240...
[22] Processing document 1200579318...
[23] Processing document 1200579405...
[24] Processing document 1200579480...
[25] Processing document 1200579529

[208] Processing document 1200614421...
[209] Processing document 1200615299...
[210] Processing document 1200615329...
[211] Processing document 1200615400...
[212] Processing document 1200615421...
[213] Processing document 1200615431...
[214] Processing document 1200615443...
[215] Processing document 1200615494...
[216] Processing document 1200615513...
[217] Processing document 1200615515...
[218] Processing document 1200615643...
[219] Processing document 1200615649...
[220] Processing document 1200615666...
[221] Processing document 1200615668...
[222] Processing document 1200615669...
[223] Processing document 1200615715...
[224] Processing document 1200615717...
[225] Processing document 1200615722...
[226] Processing document 1200615732...
[227] Processing document 1200615734...
[228] Processing document 1200615757...
[229] Processing document 1200615760...
[230] Processing document 1200615782...
[231] Processing document 1200616074...
[232] Processing document 1200616094...


[413] Processing document 1200682564...
[414] Processing document 1200682565...
[415] Processing document 1200682613...
[416] Processing document 1200682884...
[417] Processing document 1200682886...
[418] Processing document 1200682901...
[419] Processing document 1200682972...
[420] Processing document 1200683162...
[421] Processing document 1200683404...
[422] Processing document 1200683854...
[423] Processing document 1200686667...
[424] Processing document 1200686668...
[425] Processing document 1200686710...
[426] Processing document 1200686739...
[427] Processing document 1200686918...
[428] Processing document 1200686926...
[429] Processing document 1200687079...
[430] Processing document 1200687401...
[431] Processing document 1200687422...
[432] Processing document 1200687745...
[433] Processing document 1200687779...
[434] Processing document 1200689985...
[435] Processing document 1200690093...
[436] Processing document 1200690212...
[437] Processing document 1200690286...


[619] Processing document 1200730310...
[620] Processing document 1200730314...
[621] Processing document 1200735576...
[622] Processing document 1200735578...
[623] Processing document 1200735579...
[624] Processing document 1200735582...
[625] Processing document 1200735584...
[626] Processing document 1200735586...
[627] Processing document 1200735589...
[628] Processing document 1200735590...
[629] Processing document 1200735593...
[630] Processing document 1200735594...
[631] Processing document 1200735670...
[632] Processing document 1200735698...
[633] Processing document 1200735710...
[634] Processing document 1200735744...
[635] Processing document 1200735796...
[636] Processing document 1200735797...
[637] Processing document 1200735802...
[638] Processing document 1200735831...
[639] Processing document 1200735903...
[640] Processing document 1200736480...
[641] Processing document 1200736516...
[642] Processing document 1200737339...
[643] Processing document 1200740370...


[825] Processing document 1200808760...
[826] Processing document 1200808812...
[827] Processing document 1200809230...
[828] Processing document 1200809261...
[829] Processing document 1200809592...
[830] Processing document 1200811083...
[831] Processing document 1200811890...
[832] Processing document 1200812230...
[833] Processing document 1200818390...
[834] Processing document 1200822844...
[835] Processing document 1200823085...
[836] Processing document 1200825320...
[837] Processing document 1200825323...
[838] Processing document 1200825569...
[839] Processing document 1200825593...
[840] Processing document 1200826028...
[841] Processing document 1200828443...
[842] Processing document 1200828823...
[843] Processing document 1200830606...
[844] Processing document 1200830643...
[845] Processing document 1200830646...
[846] Processing document 1200830649...
[847] Processing document 1200831080...
[848] Processing document 1200831353...
[849] Processing document 1200831792...


...

[41120] Processing document 1204250789...
[41121] Processing document 1204250883...
[41122] Processing document 1204252037...
[41123] Processing document 1204252467...
[41124] Processing document 1204252793...
[41125] Processing document 1204253280...
[41126] Processing document 1204253987...
[41127] Processing document 1201855662...
[41128] Processing document 1204252470...
[41129] Processing document 1204268988...
[41130] Processing document 1204272603...
[41131] Processing document 1204269679...
[41132] Processing document 1204271507...
[41133] Processing document 1204271587...
[41134] Processing document 1204272604...
[41135] Processing document 1204274348...
[41136] Processing document 1204275899...
[41137] Processing document 1204275900...
[41138] Processing document 1204276246...
[41139] Processing document 1204274342...
[41140] Processing document 1204274685...
[41141] Processing document 1204276030...
[41142] Processing document 1204275019...
[41143] Processing document 120427

[41318] Processing document 63437667...
[41319] Processing document 63437728...
[41320] Processing document 63437729...
[41321] Processing document 63437738...
[41322] Processing document 63437746...
[41323] Processing document 63437750...
[41324] Processing document 63438161...
[41325] Processing document 63438424...
[41326] Processing document 63438628...
[41327] Processing document 63438656...
[41328] Processing document 63438859...
[41329] Processing document 63441423...
[41330] Processing document 63441777...
[41331] Processing document 63442038...
[41332] Processing document 63442263...
[41333] Processing document 63442325...
[41334] Processing document 63442609...
[41335] Processing document 63443143...
[41336] Processing document 63443606...
[41337] Processing document 63448986...
[41338] Processing document 63449367...
[41339] Processing document 63449446...
[41340] Processing document 63449590...
[41341] Processing document 63449624...
[41342] Processing document 63450068...


[41516] Processing document 1200073350...
[41517] Processing document 1200073381...
[41518] Processing document 1200073585...
[41519] Processing document 1200073618...
[41520] Processing document 1200073706...
[41521] Processing document 1200075653...
[41522] Processing document 1200075682...
[41523] Processing document 1200075689...
[41524] Processing document 1200075747...
[41525] Processing document 1200075911...
[41526] Processing document 1200075926...
[41527] Processing document 1200075963...
[41528] Processing document 1200076118...
[41529] Processing document 1200078100...
[41530] Processing document 1200078101...
[41531] Processing document 1200078102...
[41532] Processing document 1200078103...
[41533] Processing document 1200078211...
[41534] Processing document 1200078366...
[41535] Processing document 1200078400...
[41536] Processing document 1200078417...
[41537] Processing document 1200078490...
[41538] Processing document 1200079152...
[41539] Processing document 120007

[41713] Processing document 1202476624...
[41714] Processing document 1202476625...
[41715] Processing document 1202480003...
[41716] Processing document 1202480004...
[41717] Processing document 1202483733...
[41718] Processing document 1202483930...
[41719] Processing document 1202487762...
[41720] Processing document 1202487772...
[41721] Processing document 1202487773...
[41722] Processing document 1202487774...
[41723] Processing document 1202487775...
[41724] Processing document 1202489917...
[41725] Processing document 1202490182...
[41726] Processing document 1202492534...
[41727] Processing document 1202493107...
[41728] Processing document 1202493108...
[41729] Processing document 1202496272...
[41730] Processing document 1202496321...
[41731] Processing document 1202496337...
[41732] Processing document 1202496340...
[41733] Processing document 1202496341...
[41734] Processing document 1202497250...
[41735] Processing document 1202498794...
[41736] Processing document 120249

[41909] Processing document 1204356665...
[41910] Processing document 1204357103...
[41911] Processing document 1204358386...
[41912] Processing document 1204362038...
[41913] Processing document 1204362443...
[41914] Processing document 1204362559...
[41915] Processing document 1204362560...
[41916] Processing document 1204365207...
[41917] Processing document 1204368822...
[41918] Processing document 1204368823...
[41919] Processing document 1204375332...
[41920] Processing document 1204380024...
[41921] Processing document 1204386355...
[41922] Processing document 1204389058...
[41923] Processing document 1204390858...
[41924] Processing document 1204390859...
[41925] Processing document 1200393405...
[41926] Processing document 63644509...
[41927] Processing document 1201488772...
[41928] Processing document 1201495049...
[41929] Processing document 1204319044...
[41930] Processing document 63639401...
[41931] Processing document 63648852...
[41932] Processing document 63787583...


In [53]:
data[data['文档号码'].isin(error_docs)]

,文档号码,投资者关系活动主要内容介绍


In [55]:
all_scores.to_excel('readability/readability.xlsx', index=False)